In [26]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import epicstore_api
from epicstore_api import EpicGamesStoreAPI, OfferData
import json
import ast
api = EpicGamesStoreAPI()

In [66]:
df = pd.read_csv("~/Desktop/dshw/EGS_project/all.csv")
type(df["tags"][1])
df["tags"][1]

def convert_value(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        # Return None or a default value for malformed strings
        return None

# Convert the string to list of dictionaries, handling NaN values
df["taglist"] = df["tags"].apply(convert_value)
df["taglist"][1][1]["id"]
df["taglist"][1]
tag_series = []
for row in df["taglist"]:
    row_taglist = []
    if row == None:
        tag_series.append(None)
        continue
    for minidict in row:
        row_taglist.append(minidict["id"])
    # print(row_taglist)
    tag_series.append(row_taglist)
df["taglist"]
tag_series
df["tags"] = tag_series
df["tags"]
df.head()
# df.to_csv("~/Desktop/dshw/EGS_project/all_withtags.csv", index = True)


,Unnamed: 0.1,Unnamed: 0,Number,Name,Start,End,Link,id,descr,namespace,orig_price,fmt_orig_price,tags,seller,taglist
0,0,0,477,The Bridge,3/14/2024,3/21/2024,https://store.epicgames.com/en-US/p/the-bridge,6b228a1ad940496dabc89ec6640107f3,The Bridge,epic,999.0,$9.99,"[1298, 1370, 9547, 1117, 9549]",The Quantum Astrophysicists Guild,"[{'id': '1298'}, {'id': '1370'}, {'id': '9547'..."
1,1,1,476,Deus Ex: Mankind Divided,3/14/2024,3/21/2024,https://store.epicgames.com/en-US/p/deus-ex-ma...,a74220fd07e34b01987fb801433fb2e6,Secure the Digital Deluxe edition now to gain ...,5c5b2b01c8ce475d86cdd2c76fd11f7b,4499.0,$44.99,"[1336, 1370, 9547, 1084]",Eidos Interactive Corporation,"[{'id': '1336'}, {'id': '1370'}, {'id': '9547'..."
2,2,2,475,Astro Duel 2,3/7/2024,3/14/2024,https://store.epicgames.com/en-US/p/astro-duel...,bc38eac0277d41ec955690e25779cb53,Astro Duel 2 is a sci-fi combat game combining...,4a681a809a094e2c8dcc68353c68fed6,1999.0,$19.99,"[1216, 21894, 19847, 9547, 9549, 1264, 1299, 1...",NaN,"[{'id': '1216'}, {'id': '21894'}, {'id': '1984..."
3,3,3,474,Aerial_Knight’s Never Yield,2/29/2024,3/7/2024,https://store.epicgames.com/en-US/p/aerial-kni...,2fecb1793ce446408cd869611400a797,Take the role of Wally. A mysterious character...,2ed7ed64d8604265924b85bbd5670220,1199.0,$11.99,"[1216, 21894, 9547, 9549, 1263]",Headup Gmbh,"[{'id': '1216'}, {'id': '21894'}, {'id': '9547..."
4,4,4,473,Super Meat Boy Forever,2/22/2024,2/29/2024,https://store.epicgames.com/en-US/p/super-meat...,ef66029a1c0d467ea7755f6a88b4088c,Super Meat Boy Forever,epic,1999.0,$19.99,"[1216, 1370, 9547, 1151]",NaN,"[{'id': '1216'}, {'id': '1370'}, {'id': '9547'..."


In [71]:
wide = df.copy()
wide['action'] = df['tags'].apply(lambda x: "1216" in x if x is not None else False)
wide['adventure'] = df['tags'].apply(lambda x: "1117" in x if x is not None else False)
wide['puzzle'] = df['tags'].apply(lambda x: "1298" in x if x is not None else False)
wide['rpg'] = df['tags'].apply(lambda x: "1267" in x if x is not None else False)
wide['shooter'] = df['tags'].apply(lambda x: "1210" in x if x is not None else False)
wide['strategy'] = df['tags'].apply(lambda x: "1115" in x if x is not None else False)
wide['survival'] = df['tags'].apply(lambda x: "1080" in x if x is not None else False)
wide['osx'] = df['tags'].apply(lambda x: "9548" in x if x is not None else False)
wide['windows'] = df['tags'].apply(lambda x: "9547" in x if x is not None else False)
wide['singleplayer'] = df['tags'].apply(lambda x: "1370" in x if x is not None else False)
wide['multiplayer'] = df['tags'].apply(lambda x: "1203" in x if x is not None else False)
wide.tail()
# wide["action"].sum()
# wide.tail()


,Unnamed: 0.1,Unnamed: 0,Number,Name,Start,End,Link,id,descr,namespace,...,adventure,puzzle,rpg,shooter,strategy,survival,osx,windows,singleplayer,multiplayer
472,472,472,5,Axiom Verge,2/7/2019,2/22/2019,https://store.epicgames.com/en-US/p/axiom-verge,15a6c7220ba045448aeaed7e7c2f9a1f,Axiom Verge,epic,...,False,False,False,False,False,False,False,True,True,False
473,473,473,4,The Jackbox Party Pack,1/24/2019,2/7/2019,https://store.epicgames.com/en-US/p/jackbox-pa...,f2546a46804147bfb6ab74937caf9295,For 1-100 players! Your phones or tablets are ...,feverfew,...,False,False,False,False,False,False,False,True,False,True
474,474,474,3,What Remains of Edith Finch,1/11/2019,1/24/2019,https://store.epicgames.com/en-US/p/what-remai...,b1f4888a7c9b4449b6bfbace4de24a9c,What Remains of Edith Finch is a collection of...,badger,...,False,True,False,False,False,False,False,True,True,False
475,475,475,2,Super Meat Boy,12/28/2018,1/10/2019,https://store.epicgames.com/en-US/p/super-meat...,13434c6d49b742c7b195201573c146c9,Super Meat Boy,epic,...,False,False,False,False,False,False,False,True,True,False
476,476,476,1,Subnautica,12/12/2018,12/27/2018,https://store.epicgames.com/en-US/p/subnautica,a6d73af1b7de4ae086e61f462a351272,Subnautica,epic,...,True,False,False,False,False,True,True,True,True,False


In [40]:
def jprint(thing):
    print(json.dumps(thing, indent = 2, default = str))
# jprint(api.fetch_catalog_tags())
# elements = df["taglist"]['data']['Catalog']['tags']['elements']

# # Create a dictionary from the elements
# id_name_dict = {element['id']: element['name'] for element in elements}

# print(id_name_dict)
tester = df["taglist"][1]
# for minidict in df["taglist"][1]:
tester
row_taglist = []
for minidict in tester:
    row_taglist.append(minidict["id"])
row_taglist

['1336', '1370', '9547', '1084']